# MyGamePass #
## Description and Tag Preprocessing ##

A combined dataset from Steam of over 27,000 games with extensive descriptive information collected from the Steam store and Steam Spy API, packaged and published to Kaggle by Nik Davis. 

https://www.kaggle.com/nikdavis/steam-store-games

Three files will be used from this dataset.  This notebook will focus on the steam_description_data.csv and steamspy_tag_data.csv

The steam.csv master file contains the appid and store information for each game, and it will be merged with the description for each game found in the steam_description_data.csv file.  

Additionally, the steamspy_tag dataframe which includes the counts of each key term for each game will be used for content filtering.

In [1]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Read in the description dataframe and take a look
descr_df = pd.read_csv('data/steam_store/steam_description_data.csv')
descr_df.head(3)

,steam_appid,detailed_description,about_the_game,short_description
0,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...
1,20,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...
2,30,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...


In [3]:
# Compare the lengths of each description column
appid=30
print('detailed len:',int(descr_df[descr_df['steam_appid']==appid]['detailed_description'].str.len()))
print('about len:',int(descr_df[descr_df['steam_appid']==appid]['about_the_game'].str.len()))
print('short len:',int(descr_df[descr_df['steam_appid']==appid]['short_description'].str.len()))

detailed len: 428
about len: 428
short len: 309


- As we will be using Natural Language Processing, the more detailed description the better.  Keep detailed and drop the others

In [4]:
# Drop the redundant columns
descr_df.drop(['about_the_game','short_description'],axis=1,inplace=True)
# Rename the remaining columns
descr_df.rename(columns={'steam_appid':'appid','detailed_description':'description'},inplace=True)

In [5]:
descr_df.head(3)

,appid,description
0,10,Play the world's number 1 online action game. ...
1,20,One of the most popular online action games of...
2,30,Enlist in an intense brand of Axis vs. Allied ...


- We need a new dataframe of the high-level video game information
- add appid, name, percent_positive_ratings, and total_ratings, average_playtime from the steam master table

In [6]:
# Load in the master dataframe, only the columns we need
steam_df = pd.read_csv('data/steam_clean.csv',usecols=['appid','name','percent_positive_ratings','total_ratings','average_playtime'])
steam_df.head(3)

,appid,name,average_playtime,total_ratings,percent_positive_ratings
0,10,Counter-Strike,300,127873,0.973888
1,20,Team Fortress Classic,277,3951,0.839787
2,30,Day of Defeat,187,3814,0.895648


In [7]:
# Create a new games dataframe by merging the description
games_df = steam_df.merge(descr_df,on='appid')
games_df.head(3)

,appid,name,average_playtime,total_ratings,percent_positive_ratings,description
0,10,Counter-Strike,300,127873,0.973888,Play the world's number 1 online action game. ...
1,20,Team Fortress Classic,277,3951,0.839787,One of the most popular online action games of...
2,30,Day of Defeat,187,3814,0.895648,Enlist in an intense brand of Axis vs. Allied ...


In [8]:
# Output to a new csv file and read in to confirm
games_df.to_csv('data/games.csv',index=False)
newdf = pd.read_csv('data/games.csv')
newdf.head(3)

,appid,name,average_playtime,total_ratings,percent_positive_ratings,description
0,10,Counter-Strike,300,127873,0.973888,Play the world's number 1 online action game. ...
1,20,Team Fortress Classic,277,3951,0.839787,One of the most popular online action games of...
2,30,Day of Defeat,187,3814,0.895648,Enlist in an intense brand of Axis vs. Allied ...


In [9]:
newdf[newdf['name'].str.contains('Mass Effect')]

,appid,name,average_playtime,total_ratings,percent_positive_ratings,description
382,17460,Mass Effect,300,10773,0.938179,"<img src=""https://steamcdn-a.akamaihd.net/stea..."
488,24980,Mass Effect 2,300,11217,0.951680,"<img src=""https://steamcdn-a.akamaihd.net/stea..."


### Secondary Games Dataframe with additional non-numeric features from Steam master table combined ###

One strategy is to combine the non-numeric features into the description prior to Natural Language Processing so they will be included in the analysis.  We will set that alternative games dataframe up now./

In [10]:
# Read in the master dataframe with the chosen features
df = pd.read_csv('data/steam_store/steam.csv',usecols=['appid','developer','publisher','categories','steamspy_tags'])
df.head(3)

,appid,developer,publisher,categories,steamspy_tags
0,10,Valve,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action;FPS;Multiplayer
1,20,Valve,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action;FPS;Multiplayer
2,30,Valve,Valve,Multi-player;Valve Anti-Cheat enabled,FPS;World War II;Multiplayer


In [11]:
# Combine the features into one column with a separator 
df['combo'] = df['developer'] + ';' + df['publisher'] + ';' + df['categories'] + ';' + df['steamspy_tags']
df.head(3)

,appid,developer,publisher,categories,steamspy_tags,combo
0,10,Valve,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action;FPS;Multiplayer,Valve;Valve;Multi-player;Online Multi-Player;L...
1,20,Valve,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action;FPS;Multiplayer,Valve;Valve;Multi-player;Online Multi-Player;L...
2,30,Valve,Valve,Multi-player;Valve Anti-Cheat enabled,FPS;World War II;Multiplayer,Valve;Valve;Multi-player;Valve Anti-Cheat enab...


In [12]:
# Filter down to the two columns we need
df = df.filter(['appid','combo'])
df.head(3)

,appid,combo
0,10,Valve;Valve;Multi-player;Online Multi-Player;L...
1,20,Valve;Valve;Multi-player;Online Multi-Player;L...
2,30,Valve;Valve;Multi-player;Valve Anti-Cheat enab...


In [13]:
# Separate into individual words (punctuation is fine, it will be handled later)
df['combo'] = df['combo'].str.split(';')
df.head()

,appid,combo
0,10,"[Valve, Valve, Multi-player, Online Multi-Play..."
1,20,"[Valve, Valve, Multi-player, Online Multi-Play..."
2,30,"[Valve, Valve, Multi-player, Valve Anti-Cheat ..."
3,40,"[Valve, Valve, Multi-player, Online Multi-Play..."
4,50,"[Gearbox Software, Valve, Single-player, Multi..."


In [14]:
# Output this to a file
df.to_csv('data/steam_combo.csv',index=False)
newdf = pd.read_csv('data/steam_combo.csv')
newdf.head(3)

,appid,combo
0,10,"['Valve', 'Valve', 'Multi-player', 'Online Mul..."
1,20,"['Valve', 'Valve', 'Multi-player', 'Online Mul..."
2,30,"['Valve', 'Valve', 'Multi-player', 'Valve Anti..."


In [15]:
# We will be using that games_df from before, let's confirm it is ready
games_df.head(3)

,appid,name,average_playtime,total_ratings,percent_positive_ratings,description
0,10,Counter-Strike,300,127873,0.973888,Play the world's number 1 online action game. ...
1,20,Team Fortress Classic,277,3951,0.839787,One of the most popular online action games of...
2,30,Day of Defeat,187,3814,0.895648,Enlist in an intense brand of Axis vs. Allied ...


In [16]:
# Merge the two dataframes to combine all of the text data
combo_df = games_df.merge(df, on='appid')
combo_df.head(3)

,appid,name,average_playtime,total_ratings,percent_positive_ratings,description,combo
0,10,Counter-Strike,300,127873,0.973888,Play the world's number 1 online action game. ...,"[Valve, Valve, Multi-player, Online Multi-Play..."
1,20,Team Fortress Classic,277,3951,0.839787,One of the most popular online action games of...,"[Valve, Valve, Multi-player, Online Multi-Play..."
2,30,Day of Defeat,187,3814,0.895648,Enlist in an intense brand of Axis vs. Allied ...,"[Valve, Valve, Multi-player, Valve Anti-Cheat ..."


In [17]:
# Create a new combined column
combo_df['descr_combo'] = combo_df['description'] + str(combo_df['combo'])
# Drop the relic combo column
combo_df.drop('combo',1,inplace=True)
combo_df.head(3)

,appid,name,average_playtime,total_ratings,percent_positive_ratings,description,descr_combo
0,10,Counter-Strike,300,127873,0.973888,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...
1,20,Team Fortress Classic,277,3951,0.839787,One of the most popular online action games of...,One of the most popular online action games of...
2,30,Day of Defeat,187,3814,0.895648,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...


In [18]:
# Output to a file and confirm
combo_df.to_csv('data/games_combo.csv',index=False)
new_df = pd.read_csv('data/games_combo.csv')
new_df.head(3)

,appid,name,average_playtime,total_ratings,percent_positive_ratings,description,descr_combo
0,10,Counter-Strike,300,127873,0.973888,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...
1,20,Team Fortress Classic,277,3951,0.839787,One of the most popular online action games of...,One of the most popular online action games of...
2,30,Day of Defeat,187,3814,0.895648,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...


### Fantastic!  We are ready for content based filtering ###